In [3]:
!pip install python-twitter

  Using cached https://files.pythonhosted.org/packages/b3/a9/2eb36853d8ca49a70482e2332aa5082e09b3180391671101b1612e3aeaf1/python_twitter-3.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/94/e7/c250d122992e1561690d9c0f7856dadb79d61fd4bdd0e598087dce607f6c/requests_oauthlib-1.0.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 143kB 9.0MB/s ta 0:00:01   14% |████▋                           | 20kB 2.6MB/s eta 0:00:01


In [4]:
!pip install psycopg2-binary

  Using cached https://files.pythonhosted.org/packages/cd/eb/4e872a11edd82079b4163035389051668c58cd2acc30777b6bee73f5f8a3/psycopg2_binary-2.7.6.1-cp36-cp36m-manylinux1_x86_64.whl


In [5]:
import twitter

In [6]:

import json
import time
import psycopg2 as pg2
import numpy as np
import pandas as pd

from datetime import datetime
from psycopg2.extras import RealDictCursor, Json

In [7]:
%run sql_test.py

In [8]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    ''' 
    Returns both a connection and a cursor object for your database
    '''

    con = pg2.connect(host=IP_ADDRESS, #allows you to navigate db
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    '''
    Executes a query directly to a database, without having to create a cursor and connection each time. 
    '''
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

def insert_entry_json(data, tablename=None):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
    con.commit()
    con.close()

In [9]:
twitter_keys = {
    'consumer_key':        'RlBCfTfpKVtm1sM1hRLynGS9e',
    'consumer_secret':     'jvbloXLzGmoWAkFo3TrxJrOHRTj0Js45o9utubpGJLzKJchbZw',
    'access_token_key':    '862547486-ggaabuCL7qQzE2iMmizOTVIrFE1YAJAAoPX9b74a',
    'access_token_secret': 'IMR8xU0QEbKTYtt3QPfa0UP5daJTiHyM8IDSr7rlIzqT5'
}

api = twitter.Api(consumer_key         =   twitter_keys['consumer_key'],
                  consumer_secret      =   twitter_keys['consumer_secret'],
                  access_token_key     =   twitter_keys['access_token_key'],
                  access_token_secret  =   twitter_keys['access_token_secret'],
)

In [10]:
print(api.VerifyCredentials())

{"created_at": "Fri Oct 05 05:14:58 +0000 2012", "default_profile": true, "description": "Climber, Writer, Traveler, and then some.", "followers_count": 42, "friends_count": 35, "geo_enabled": true, "id": 862547486, "id_str": "862547486", "lang": "en", "name": "Chris Sinatra", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_banner_url": "https://pbs.twimg.com/profile_banners/862547486/1353964028", "profile_image_url": "http://pbs.twimg.com/profile_images/2683801931/62818c8d7da1626b1dbb97b09995b1cd_normal.jpeg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/2683801931/62818c8d7da1626b1dbb97b09995b1cd_normal.jpeg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "s

In [ ]:
def streamTweets(city, geocode, before, count, sql_db):
    for i in range(1,8):
        year, month, day = before.split('-')
        day = int(day)
        day-=1
        day = str(day).zfill(2)
        date = year + month + day
        after = datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
        
        results = api.GetSearch(
#             before = before,
#             terms =
            geocode = geocode,
            return_json = True
        )

        insert_entry_json(results['statuses'], sql_db)
        before = after

In [ ]:
def tweet_repeater(city, geocode, start, repeats=15, count=100, sql_db='raw_tweets'):
    for i in range(repeats):
        before = start
        
        streamTweets(city, geocode, before, count, sql_db)
        print(f'Loop {i+1} complete. Raw tweets pushed to {sql_db}.')
        time.sleep(450)
        
    print('All tweets pulled.')

In [ ]:
tweet_repeater(city='cha', geocode='35.2271,-80.8431,10mi', start='2019-01-08', repeats=100, count=100, sql_db='raw_tweets')

Loop 1 complete. Raw tweets pushed to raw_tweets.
Loop 2 complete. Raw tweets pushed to raw_tweets.
